Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

from datetime import datetime as dt

In [3]:
data_root = '../../input/notMNIST/' # Change me to store data elsewhere
pickle_file = data_root + 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)


Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np


def reformat(dataset, labels):
    dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels


train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    # Input data.
    # [16, 28, 28, 1]
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    # [16, 10]
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    # [10000, 28, 28, 1]
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables. 
    # [5, 5, 1, 16]
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1, seed=1))
    
    # [16]
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    # [5, 5, 16, 16]
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1, seed=1))
    # [16]
    layer2_biases = tf.Variable(tf.constant(0.0, shape=[depth]))
    
    # [28//4 * 28//4 * 16, 64] due to Padding is "SAME"
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1, seed=1))
    # [64]
    layer3_biases = tf.Variable(tf.constant(0.0, shape=[num_hidden]))
    # [64, 10]
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1, seed=1))
    # [10]
    layer4_biases = tf.Variable(tf.constant(0.0, shape=[num_labels]))

    tf.summary.histogram("layer1_weights", layer1_weights)
    tf.summary.histogram("layer1_biases", layer1_biases)
    tf.summary.histogram("layer2_weights", layer2_weights)
    tf.summary.histogram("layer2_biases", layer2_biases)
    tf.summary.histogram("layer3_weights", layer3_weights)
    tf.summary.histogram("layer3_biases", layer3_biases)
    tf.summary.histogram("layer4_weights", layer4_weights)
    tf.summary.histogram("layer4_biases", layer4_biases)

    # Model.
    def model(data, name):
        with tf.name_scope("Conn_hidden_1_" + name):
            conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer1_biases)
            
            tf.summary.scalar('output_sum', tf.reduce_sum(hidden) )
            
        with tf.name_scope("Conn_hidden_2_" + name):
            conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer2_biases)
            shape = hidden.get_shape().as_list()
            reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
            print("shape: ", shape)
            tf.summary.scalar('output_sum', tf.reduce_sum(reshape))
       
        # need to reshape to match the datashape between CNN and FC. 
        
        with tf.name_scope("FC_hidden_3_" + name):
            hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
            tf.summary.scalar('output_sum', tf.reduce_sum(hidden))
        
        with tf.name_scope("FC_output_" + name):
            logits = tf.matmul(hidden, layer4_weights) + layer4_biases
            tf.summary.scalar('output_sum', tf.reduce_sum(logits))
            
        return logits


    # Training computation.
    logits = model(tf_train_dataset, name='')
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset, name='V'))
    test_prediction = tf.nn.softmax(model(tf_test_dataset, name='T'))

shape:  [16, 7, 7, 16]
shape:  [10000, 7, 7, 16]
shape:  [10000, 7, 7, 16]


In [7]:
num_steps = 1001
log_steps = 50

with tf.Session(graph=graph) as session:
    # variable initialization
    tf.global_variables_initializer().run()
    print('Initialized')
    
    # Make the tensorboard log writer
    # Merge all the tf summary
    session_log_dir = "logs/4_1/sample1/" + dt.today().strftime('%m%d_%H%M')
    writer = tf.summary.FileWriter(session_log_dir)
    writer.add_graph(session.graph)
    merged_summary = tf.summary.merge_all()
    print("Logging Directory : %s" % session_log_dir)
    
    for step in range(num_steps):
        # Prepare the dataset
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        # Run Summary Log Writer
        if step % log_steps == 0:
            s = session.run(merged_summary, feed_dict=feed_dict)
            writer.add_summary(s, step)
        
        # Run TF objects
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        # Run accuracy and print out
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    
    # Run accuracy and print out
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Logging Directory : logs/4_1/sample1/1031_0938
Minibatch loss at step 0: 2.316378
Minibatch accuracy: 18.8%
Validation accuracy: 13.8%
Minibatch loss at step 50: 1.237840
Minibatch accuracy: 68.8%
Validation accuracy: 60.2%
Minibatch loss at step 100: 0.914941
Minibatch accuracy: 68.8%
Validation accuracy: 75.7%
Minibatch loss at step 150: 0.478011
Minibatch accuracy: 87.5%
Validation accuracy: 76.0%
Minibatch loss at step 200: 1.093380
Minibatch accuracy: 68.8%
Validation accuracy: 77.7%
Minibatch loss at step 250: 1.165090
Minibatch accuracy: 68.8%
Validation accuracy: 78.5%
Minibatch loss at step 300: 0.430676
Minibatch accuracy: 87.5%
Validation accuracy: 80.0%
Minibatch loss at step 350: 0.484693
Minibatch accuracy: 93.8%
Validation accuracy: 78.9%
Minibatch loss at step 400: 0.301811
Minibatch accuracy: 93.8%
Validation accuracy: 80.7%
Minibatch loss at step 450: 0.897717
Minibatch accuracy: 81.2%
Validation accuracy: 80.0%
Minibatch loss at step 500: 0.808585
Minibat

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---